In [ ]:
application "tropical";

In [ ]:
sub harnack_curve_tmp {
  my $n = shift;
  my @monoms;
  my @weights;
  my @signs;
  foreach my $i (0 .. $n) {
    foreach my $j (0 .. $n-$i) {
      push @weights, ($i*$j+$i*$i+$j*$j);
      push @monoms, [$n-$i-$j, $i, $j];
      push @signs, ($i*$j+$i+$j)%2;
    }
  }

  my $m = new Matrix<Int>(\@monoms);
  my $c = new Vector<TropicalNumber<Min>>(\@weights);
  my $s = new Array<Bool>(\@signs);
  my $h = new Hypersurface<Min>(MONOMIALS=>$m, COEFFICIENTS=>$c);
  $h->PATCHWORK(SIGNS=>$s);
  return $h;
}

In [ ]:
$h = harnack_curve_tmp(5);

In [ ]:
$h->properties;

In [ ]:
$comp = $h->COMPACTIFICATION;

In [ ]:
$rf = $h->PATCHWORK->REAL_FACETS;

In [ ]:
print $rf;

In [ ]:
print rows_labeled($h->MAXIMAL_POLYTOPES);

In [ ]:
print rows_labeled($h->COMPACTIFICATION->DECORATION);

In [ ]:
print $comp->nodes_of_rank(2)->size();

In [ ]:
$decor = $comp->DECORATION;
$facets = $h->MAXIMAL_POLYTOPES;
$real_facets = $h->PATCHWORK->REAL_FACETS;
$nm = new NodeMap<Directed, Set<Int>>($comp->ADJACENCY);
foreach my $node (@{$comp->nodes_of_rank(2)}){
    my $real = $decor->[$node]->realisation;
    # Only works since the thing is pure
    my $facetno = -1;
    for(my $i=0; $i<$facets->rows; $i++){
        if($facets->[$i] == $real){
            $facetno = $i;
            last;
        }
    }
    print $node," ",$real," ",$facetno,": ";

    for(my $i = 0; $i<$real_facets->rows; $i++){
        if($real_facets->[$i]->contains($facetno)){
            print $i,",";
            $nm->[$node] += $i;
        }
    }
    print "\n";
    
}

In [ ]:
foreach my $node (@{$comp->nodes_of_rank(1)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}
foreach my $node (@{$comp->nodes_of_rank(0)}){
    # print $comp->ADJACENCY->out_adjacent_nodes($node),"\n";
    foreach my $parent (@{$comp->ADJACENCY->out_adjacent_nodes($node)}){
        $nm->[$node] += $nm->[$parent];
    }
}

In [ ]:
$em = new EdgeMap<Directed, Matrix<GF2>>($comp->ADJACENCY);

In [ ]:
sub make_matrix {
    my($source, $target) = @_;
    # print "S: ",$source,"T: ",$target,"\n";
    my $result = unit_matrix<GF2>($source->size());
    my $remaps = new Set<Int>();
    my $remapt = new Set<Int>();
    my $i = 0;
    foreach my $s (@$source){
        $remaps += $i;
        if($target->contains($s)){
            $remapt += $i;
        }
        $i++;
    }
    return $result->minor($remapt, $remaps);
}

In [ ]:
for (my $e=entire(edges($comp->ADJACENCY)); $e; ++$e) {
    $em->[$$e] = make_matrix($nm->[$e->from_node], $nm->[$e->to_node]);
}

In [ ]:
print $comp->TOP_NODE;

In [ ]:
$top = $comp->TOP_NODE;
foreach my $node (@{$comp->ADJACENCY->in_adjacent_nodes($top)}){
    $em->edge($node, $top) = zero_matrix<GF2>($nm->[$node]->size(),0);
}

In [ ]:
$Polymake::User::Verbose::cpp = 1;
$chain = fan::build_full_chain($comp, $comp->ORIENTATIONS, $em, true);

In [ ]:
print $comp->ORIENTATIONS;

In [ ]:
print $chain;

In [ ]:
print $chain->type->full_name;

In [ ]:
print topaz::betti_numbers<GF2>($chain);

In [ ]:
$hr = $h->PATCHWORK->realize("uniform");

In [ ]:
print $hr->HOMOLOGY;